In [ ]:
%load_ext autoreload
%autoreload 2

# Score climatology

Compute and score the climatology, to use as a baseline.

In [ ]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import datetime
import pathlib
import xarray as xr

In [ ]:
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***HOME***hdd_scratch/s2s/training-input/0.3.0/netcdf'

## Boost dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
)

In [ ]:
cluster.scale(jobs=3)  # Scale to two working nodes as configured.
client = dask.distributed.Client(cluster)

In [ ]:
client

# Open dataset

In [ ]:
CENTER = 'ecmwf'
FIELD = 't2m'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
files[:10]

In [ ]:
dataset = xr.open_mfdataset(files, preprocess=fix_dataset_dims)

In [ ]:
dataset

In [ ]:
valid_time = dataset.valid_time.compute()

In [ ]:
valid_time[0, 0]